## Interpreting Feature Importance Using Logistic Regression

In this section, we use logistic regression to analyze the importance of different student-related features in predicting placement outcomes. Logistic regression models the log-odds of a binary outcome as a linear combination of input features, allowing direct interpretation of feature coefficients.


In [1]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler


In [2]:
df = pd.read_csv("full_dataset.csv")
df.head()


,Student_ID,Age,Gender,Degree,Branch,CGPA,Internships,Projects,Coding_Skills,Communication_Skills,Aptitude_Test_Score,Soft_Skills_Rating,Certifications,Backlogs,Placement_Status
0,1048,22,Female,B.Tech,ECE,6.29,0,3,4,6,51,5,1,3,Not Placed
1,37820,20,Female,BCA,ECE,6.05,1,4,6,8,59,8,2,1,Not Placed
2,49668,22,Male,MCA,ME,7.22,1,4,6,6,58,6,2,2,Not Placed
3,19467,22,Male,MCA,ME,7.78,2,4,6,6,90,4,2,0,Placed
4,23094,20,Female,B.Tech,ME,7.63,1,4,6,5,79,6,2,0,Placed


### Logistic Regression Overview

Logistic regression models the probability of a binary outcome using the following relationship:

\[
\log \left(\frac{p}{1 - p}\right) = \beta_0 + \beta_1 x_1 + \beta_2 x_2 + \cdots + \beta_k x_k
\]

Each coefficient represents the change in the log-odds of being placed associated with a one-unit increase in the corresponding feature, holding all other features constant.


In [3]:
y = df["Placement_Status"]

X = df.drop(columns=["Placement_Status", "Student_ID"])


In [4]:
X.dtypes

Age                       int64
Gender                   object
Degree                   object
Branch                   object
CGPA                    float64
Internships               int64
Projects                  int64
Coding_Skills             int64
Communication_Skills      int64
Aptitude_Test_Score       int64
Soft_Skills_Rating        int64
Certifications            int64
Backlogs                  int64
dtype: object

### Feature Preprocessing

The dataset contains both numerical and categorical variables. Since logistic regression requires numerical inputs, categorical variables such as Gender, Degree, and Branch are converted into binary indicators using one-hot encoding.

The identifier column `Student_ID` is removed because it does not contain predictive information.


In [5]:
X_encoded = pd.get_dummies(X, drop_first=True)
X_encoded

,Age,CGPA,Internships,Projects,Coding_Skills,Communication_Skills,Aptitude_Test_Score,Soft_Skills_Rating,Certifications,Backlogs,Gender_Male,Degree_B.Tech,Degree_BCA,Degree_MCA,Branch_Civil,Branch_ECE,Branch_IT,Branch_ME
0,22,6.29,0,3,4,6,51,5,1,3,False,True,False,False,False,True,False,False
1,20,6.05,1,4,6,8,59,8,2,1,False,False,True,False,False,True,False,False
2,22,7.22,1,4,6,6,58,6,2,2,True,False,False,True,False,False,False,True
3,22,7.78,2,4,6,6,90,4,2,0,True,False,False,True,False,False,False,True
4,20,7.63,1,4,6,5,79,6,2,0,False,True,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,19,4.91,0,1,1,5,47,5,0,3,False,False,False,True,True,False,False,False
49996,20,6.82,2,3,5,7,76,6,2,0,True,True,False,False,False,True,False,False
49997,20,5.80,0,3,6,7,56,6,2,2,True,False,True,False,False,False,True,False
49998,23,7.67,1,4,7,6,67,4,2,0,False,True,False,False,False,False,True,False


### Feature Scaling

All features are standardized using z-score normalization. Standardization ensures that coefficient magnitudes are directly comparable and reflect true relative importance rather than differences in feature scale.


In [6]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_encoded)
X_scaled


array([[ 0.50124967, -0.71654265, -0.9151296 , ...,  1.99625772,
        -0.49946869, -0.50118722],
       [-0.50124967, -0.95786977,  0.26760569, ...,  1.99625772,
        -0.49946869, -0.50118722],
       [ 0.50124967,  0.21859994,  0.26760569, ..., -0.50093733,
        -0.49946869,  1.99526237],
       ...,
       [-0.50124967, -1.20925219, -0.9151296 , ..., -0.50093733,
         2.00212749, -0.50118722],
       [ 1.00249934,  0.67108829,  0.26760569, ..., -0.50093733,
         2.00212749, -0.50118722],
       [ 1.50374901, -0.32438608,  1.45034097, ..., -0.50093733,
        -0.49946869, -0.50118722]], shape=(50000, 18))

In [7]:
log_reg = LogisticRegression(max_iter=1000)
log_reg.fit(X_scaled, y)


,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,1000
,multi_class,'deprecated'


### Interpreting Logistic Regression Coefficients

The learned coefficients from the logistic regression model are analyzed to determine feature importance. Features with larger absolute coefficient values have a stronger influence on placement probability.

Odds ratios are computed by exponentiating the coefficients to provide a more intuitive interpretation.


In [8]:
coef_df = pd.DataFrame({
    "Feature": X_encoded.columns,
    "Coefficient": log_reg.coef_[0]
})

coef_df["Absolute_Coefficient"] = coef_df["Coefficient"].abs()
coef_df["Odds_Ratio"] = np.exp(coef_df["Coefficient"])

coef_df = coef_df.sort_values(by="Absolute_Coefficient", ascending=False)
coef_df


,Feature,Coefficient,Absolute_Coefficient,Odds_Ratio
9,Backlogs,-1.805292,1.805292,0.164426
5,Communication_Skills,1.630633,1.630633,5.107107
8,Certifications,0.904727,0.904727,2.471258
1,CGPA,0.389302,0.389302,1.475951
4,Coding_Skills,0.355317,0.355317,1.426632
3,Projects,0.271343,0.271343,1.311725
6,Aptitude_Test_Score,0.198131,0.198131,1.219122
15,Branch_ECE,0.126139,0.126139,1.134440
2,Internships,0.110697,0.110697,1.117057
16,Branch_IT,0.096785,0.096785,1.101623


### Feature Importance Analysis

The most influential feature in predicting placement is **Backlogs**, which has the largest absolute coefficient and a strong negative effect. Its odds ratio of approximately 0.16 indicates that an increase in backlogs significantly reduces the likelihood of placement.

**Communication Skills** is the strongest positive predictor, with an odds ratio greater than 5. This suggests that students with stronger communication skills are substantially more likely to be placed.

Other positively contributing features include **Certifications**, **CGPA**, and **Coding Skills**, all of which align with real-world hiring expectations emphasizing academic performance and job readiness.

Categorical variables such as Degree and Branch exhibit relatively small coefficients, indicating that once skills and performance metrics are considered, academic background plays a lesser role in placement outcomes.


### Why Some Variables Are More Important

Variables such as Backlogs, Communication Skills, and Certifications are more important because they directly capture employability and academic consistency. These features provide strong, independent signals that employers consider during recruitment.

In contrast, demographic and categorical variables contribute less unique information once performance-related factors are included in the model. Logistic regression naturally emphasizes features that reduce uncertainty in predicting the outcome, resulting in larger coefficients for variables with higher predictive power.
